In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
import os
from tensorflow.keras.preprocessing import image
import numpy as np


#  Define paths
base_dir = 'DeepFake Detection'
train_dir = os.path.join(base_dir, 'train-20250112T065955Z-001\\train')
test_dir = os.path.join(base_dir, 'test-20250112T065939Z-001\\test')

#  Data preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

# Load ResNet50 base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model

#  Build the model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, validation_data=test_generator, epochs=10)

# Evaluate the model
loss, acc = model.evaluate(test_generator)
print(f"Test Accuracy: {acc:.2f}")

# Predict a model
img_path = r'C:\Users\pinka\Digicrome\fake-2.jpg'  # Change to your image
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize

prediction = model.predict(img_array)
label = 'real' if prediction[0][0] < 0.5 else 'fake'
print(f"Predicted label: {label}")



Found 479 images belonging to 2 classes.
Found 499 images belonging to 2 classes.
Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 150s 7s/step - accuracy: 0.6237 - loss: 0.6853 - val_accuracy: 0.2248 - val_loss: 0.7696
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 130s 7s/step - accuracy: 0.5968 - loss: 0.6779 - val_accuracy: 0.2242 - val_loss: 0.9385
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 127s 7s/step - accuracy: 0.6687 - loss: 0.6497 - val_accuracy: 0.2189 - val_loss: 1.0379
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 131s 7s/step - accuracy: 0.6631 - loss: 0.6348 - val_accuracy: 0.2229 - val_loss: 1.1129
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 130s 7s/step - accuracy: 0.6785 - loss: 0.6459 - val_accuracy: 0.2189 - val_loss: 0.8615
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 136s 8s/step - accuracy: 0.6757 - loss: 0.6318 - val_accuracy: 0.2175 - val_loss: 1.0031
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 142s 8s/step - accuracy: 0.6884 - loss: 0.6296 - val_accuracy: 0.2189 - val_loss: 0.8837
Epoch 8/10
15/15 ━━━━━━━━━━━━

In [5]:

# Predict a model
img_path = r'C:\Users\pinka\Digicrome\real-1.jpeg'  # Change to your image
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize

prediction = model.predict(img_array)
label = 'real' if prediction[0][0] < 0.5 else 'fake'
print(f"Predicted label: {label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Predicted label: fake
